In [3]:
import os
import glob
import csv
import re
from collections import Counter

import pandas as pd  # 用于读取 xlsx

# ========= 配置区域 =========
# 指定存放 XLSX 的文件夹路径（按你的实际路径修改）
DATA_DIR = r"../shop-data/shop10/"

# 输出文件
TEMPLATE_SUMMARY_CSV = "templates_summary_shop10.csv"
PARSED_RULES_CSV = "parsed_color_rules_shop10.csv"

# 已知颜色列表（可以根据你的数据慢慢补充）
COLORS = [
    "Blue", "Gold", "Black", "Silver", "Red", "Green", "Pink",
    "全色",  # 特殊含义：所有颜色
    "ブルー", "ゴールド", "ブラック", "シルバー", "レッド", "グリーン", "ピンク"
]

# 金额正则：匹配 2000円 / 2,000円 / ¥117,000 / ￥117,000 等
PRICE_RE = re.compile(r"[¥￥]?\s*([+-]?\d[\d,]*)\s*円?")


def find_xlsx_files(data_dir):
    pattern = os.path.join(data_dir, "*.xlsx")
    return glob.glob(pattern)


def normalize_line(line):
    """
    将一行文本中：
      - 颜色替换为 <COLOR>
      - 金额替换为 <PRICE>
    返回模板字符串
    """
    s = line.strip()
    if not s:
        return ""

    # 颜色替换：先按长度排序，避免短颜色名截断长颜色名
    sorted_colors = sorted(COLORS, key=len, reverse=True)
    for c in sorted_colors:
        if c and c in s:
            s = s.replace(c, "<COLOR>")

    # 金额替换
    s = PRICE_RE.sub("<PRICE>", s)

    return s


def parse_color_delta_line(line):
    """
    尝试解析类似：
      Blue-2000円
      Gold+2000円
      ブルー+2,000円
    成为结构化：
      { "color": xxx, "delta": +/-int, "amount": int, "sign": ±1 }

    返回 dict 或 None
    """
    s = line.strip()
    if not s:
        return None

    # 排除“全色”这种行
    if "全色" in s:
        return None

    # 可能用到的分隔符
    delimiters = ["-", "＋", "+", "−"]

    delim_pos = [(d, s.find(d)) for d in delimiters if s.find(d) > 0]
    if not delim_pos:
        return None

    # 找到最靠前的分隔符
    delim, idx = sorted(delim_pos, key=lambda x: x[1])[0]
    color_part = s[:idx].strip()
    price_part = s[idx + 1 :].strip()

    if not color_part:
        return None

    # 提取金额
    m = PRICE_RE.search(price_part)
    if not m:
        return None

    amount_str = m.group(1)
    try:
        amount = int(amount_str.replace(",", ""))
    except ValueError:
        return None

    # 根据分隔符判断正负方向（你可以按业务语义调整）
    if delim in ["-", "−"]:
        sign = -1
    else:
        sign = 1

    delta = sign * amount

    return {
        "color": color_part,
        "delta": delta,
        "amount": amount,
        "sign": sign,
    }


def detect_rule(line, template):
    """
    根据原始行和模板判断属于哪一种规则。
    这里先简单处理两类：
      - RULE_ALL_COLORS_SAME: 含“全色”
      - RULE_COLOR_DELTA_SIMPLE: 能解析出颜色 ± 金额
    返回 (rule_id, parsed_info_dict_or_None)
    """
    s = line.strip()

    # 规则1：全色同价
    if "全色" in s:
        return "RULE_ALL_COLORS_SAME", {"all_colors_same": True}

    # 规则2：颜色 + 差额
    parsed = parse_color_delta_line(s)
    if parsed:
        return "RULE_COLOR_DELTA_SIMPLE", parsed

    return None, None


def safe_str(v):
    """把 Excel 单元格值安全转成字符串，如果是 NaN 则返回空字符串。"""
    if pd.isna(v):
        return ""
    return str(v).strip()


def main():
    xlsx_files = find_xlsx_files(DATA_DIR)
    if not xlsx_files:
        print(f"在目录 {DATA_DIR} 下没有找到 XLSX 文件。")
        return

    print(f"发现 {len(xlsx_files)} 个 XLSX 文件。")

    # 收集所有行及模板
    raw_lines = []  # 每个元素: dict(file, model, time, line, template, rule_id, rule_data)
    template_counter = Counter()

    for path in xlsx_files:
        print(f"处理文件: {path}")
        # 读取 Excel，默认第一张表（如果有多表可以加 sheet_name 参数控制）
        df = pd.read_excel(path)

        for _, row in df.iterrows():
            model = safe_str(row.get("モデルナンバー"))
            note = safe_str(row.get("備考1"))
            time_scraped = safe_str(
                row.get("time-scraped")
                or row.get("time_scraped")
                or row.get("time")
            )

            if not note:
                continue

            # 按行拆分備考1
            for line in note.splitlines():
                line = line.strip()
                if not line:
                    continue
                # 过滤掉明显与颜色规则无关的行（可以根据需要调优）
                if line.startswith("※開封品"):
                    continue

                template = normalize_line(line)
                if not template:
                    continue

                template_counter[template] += 1

                rule_id, rule_data = detect_rule(line, template)

                raw_lines.append(
                    {
                        "file": os.path.basename(path),
                        "model": model,
                        "time_scraped": time_scraped,
                        "line": line,
                        "template": template,
                        "rule_id": rule_id or "",
                        "rule_data": rule_data or {},
                    }
                )

    # ===== 输出模板频率表 =====
    print(f"共收集到 {len(raw_lines)} 条候选行。")
    print(f"不同模板数量: {len(template_counter)}")

    with open(TEMPLATE_SUMMARY_CSV, "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["template", "count"])
        for tmpl, cnt in template_counter.most_common():
            writer.writerow([tmpl, cnt])

    print(f"模板频率表已写入: {TEMPLATE_SUMMARY_CSV}")

    # ===== 输出已解析的规则结果 =====
    with open(PARSED_RULES_CSV, "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(
            [
                "file",
                "model",
                "time_scraped",
                "original_line",
                "template",
                "rule_id",
                "color",
                "delta",
                "amount",
                "sign",
                "all_colors_same",
            ]
        )

        for r in raw_lines:
            rd = r["rule_data"] or {}
            writer.writerow(
                [
                    r["file"],
                    r["model"],
                    r["time_scraped"],
                    r["line"],
                    r["template"],
                    r["rule_id"],
                    rd.get("color", ""),
                    rd.get("delta", ""),
                    rd.get("amount", ""),
                    rd.get("sign", ""),
                    rd.get("all_colors_same", ""),
                ]
            )

    print(f"解析后的规则结果已写入: {PARSED_RULES_CSV}")
    print("完成。")


if __name__ == "__main__":
    main()


发现 2879 个 XLSX 文件。
处理文件: ../shop-data/shop10/00036271311-2025-11-23-shop10.xlsx
处理文件: ../shop-data/shop10/00035869365-2025-11-11-shop10.xlsx
处理文件: ../shop-data/shop10/00036535964-2025-12-01-shop10.xlsx
处理文件: ../shop-data/shop10/00035875380-2025-11-11-shop10.xlsx
处理文件: ../shop-data/shop10/00035799620-2025-11-09-shop10.xlsx
处理文件: ../shop-data/shop10/00036249907-2025-11-22-shop10.xlsx
处理文件: ../shop-data/shop10/00036261382-2025-11-22-shop10.xlsx
处理文件: ../shop-data/shop10/00036283523-2025-11-23-shop10.xlsx
处理文件: ../shop-data/shop10/00035997892-2025-11-15-shop10.xlsx
处理文件: ../shop-data/shop10/00036530890-2025-11-30-shop10.xlsx
处理文件: ../shop-data/shop10/00036294494-2025-11-23-shop10.xlsx
处理文件: ../shop-data/shop10/00036077147-2025-11-17-shop10.xlsx
处理文件: ../shop-data/shop10/00036131021-2025-11-19-shop10.xlsx
处理文件: ../shop-data/shop10/00035880438-2025-11-11-shop10.xlsx
处理文件: ../shop-data/shop10/00036549183-2025-12-01-shop10.xlsx
处理文件: ../shop-data/shop10/00035756304-2025-11-08-shop10.xlsx
处理文件: